In [ ]:
using Pkg; Pkg.activate(".")
import Random: rand!

In [ ]:
# https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/
# 
@show typeof(1)
@show Sys.WORD_SIZE
@show typemin(Int32), typemax(Int32)
@show eps(Float32)

In [ ]:
# https://docs.julialang.org/en/v1/manual/mathematical-operations/
# 
@show 3 * 2 / 12
@show [1,2,3].^2

# dot syntax

A = rand(2,3)
@show 2 .* A .^2 .+ sin.(A)
@show @. 2A^2 + sin(A)

⊗(A,B) = kron(A,B)
@show size.([A,A] .⊗ [A,A])  # computes [A⊗A, A⊗A]

@show isequal(1,1.)
@show isinf(Inf)

# chained comparisons
@show 1 < 2 <= 2 < 3 == 3 > 2 >= 1 == 1 < 3 != 5
A = [1,2,3]
@show 0 .< A .< 4

# conversions
@show Int8(127)
@show Int8(127.0)
@show round(0.50)
@show round(0.51)


# arithmetics 
@show sign(.2)
@show exp(1)
@show log(ℯ)

In [ ]:
# https://docs.julialang.org/en/v1/manual/strings/
#

# Char
@show typeof('x')  # Char: 32-bit primitive
@show Int('x')
@show Char(120)

# String
s = "Hello, world"
@show s[1], s[end-1], s[end]
@show length(s), lastindex(s)
@show s[1:5], SubString(s,1,5)
@show collect(eachindex(s))  # iterator
@show string(s, "  ", s)

# interpolation 
@show "$s from me!"
@show "1 + 2 = $(1+2)"

@show """ 
multiple
lines
"""

@show findfirst(isequal('e'), s)
@show occursin("Hello", s)

In [ ]:
# https://docs.julialang.org/en/v1/manual/functions/
# 
function f(x,y)
    x + y
end

Σ = f

@show f
@show f(2,3), Σ(2,3)

# argment passing behavior: 
#     pass-by-sharing, values not copied, args act as new variable 
#     bindnigs, values they refer to are identical to passed values
#     modification to mutable values (`Array`) is visible to caller, i.e. python

# return type
function g(x,y)::Int8
    x * y
end

@show typeof(g(2,3))

@show 1+2+3, +(1,2,3)

# special ops 
A = [1,2]
B = [3,4]

@show size(A)      # (2,) col vector
@show [A B], hcat(A,B), size([A B])
@show [A;B], vcat(A,B), size([A;B])
@show A', size(A') # (1,2) row vector
@show B[1]

# anonymous functions
@show map(x -> x^2 + 2x - 1, [1,2,3])
@show get(()-> println(time()), Dict("a"=>1, "b"=>2), "a")

# Tuple: immutable
@show (0, "hello", 2*3)
named_tuple = (a=1, b=2)
@show named_tuple, named_tuple.a

# argument destructuring
range((min,max)) = max-min
@show range((2,4))

# varargs
bar(a,b,x...) = (a,b,x)
@show bar(1,2)
@show bar(1,2,3)
@show bar(1,2,3,4)

# optional args
add1(x, optional=1) = x + optional
@show add1(2)

# keyword args
add1(x; keyword=1) = x + keyword
@show add1(2; keyword=1)

#type 
add1(x; c::Int=1) = x + c
@show add1(2)

# do-block
map([-1,0,22]) do x
    if x < 0
        return 0
    elseif x == 0
        return 1
    else
        return x
    end
end

# function composition and pipes
@show (sqrt ∘ +)(3,6)
@show map(first ∘ reverse ∘ uppercase, split("you can compose functions like this"))
@show 1:10 |> sum |> x -> x+1
@show ["list", "of", "strings"] .|> [uppercase, titlecase, length]

# dot syntax
@show sin.([1,2,3])
# # f.(args, ...) equiv to broadcast(f, args...)
f(x,y) = 3x + 4y
@show f.(pi, [1,2,3])
@show f.([2,3,4],[1,2,3])
# preallocating output array for faster code!
Y = [1.,2.,3.,4.]
X = similar(Y)
@show @. X = sin(cos(Y))

 

In [ ]:
# https://docs.julialang.org/en/v1/manual/control-flow/
#

# compound expression 
z = (x=1; y=2; x+y)
@show z

# if-elseif-else equiv to ternary operator ?:
print(1>2 ? "false" : "true")

# short-circuit
#     for short if statements
#     <cond> && <statement> evaluate if <cond> is true
#     <cond> || <statement> evaluate if <cond> is false
n = 2
n >= 0 || error("must be >= 0")

# loops
println()
for i = 1:5
    println(i)
end

for i = 1:2, j = 3:4
    println((i,j))
end

# task/coroutines
#    ̇ interruptable by another task, and resumable later, at point where it left off
#   ⋅ no space allocation on call stack
#   ⋅ order of task call not deterministic
# Channel
#   ⋅ waitable FIFO queue, w/ multiple tasks reading/writing 
function producer(c::Channel)
    # multiple return-values with put!
    put!(c, "start")
    for n = 1:4
        put!(c, 2n)
    end
    put!(c, "stop")
end

chn = Channel(producer)
@show [take!(chn) for _ in 1:6]
# channel closed when task terminates !


In [ ]:
# https://docs.julialang.org/en/v1/manual/types/
#
# Julia is dynamically typed
# All values are objects, but functions are not bundled with objects

# composite types
#     immutable (more efficient, enforce invariance)
#         ⋅ may contain Array as fields, which are mutable
#         ⋅ fields cannot be changed to point to different objects
#         ⋅ can be copied freely by compiler! no distinction between original & copy
#     mutable with `mutable struct`

struct Foo
   bar
   baz::Int
   qux::Float64
end

# construtor
#     ⋅ default ctor, accepts args and call `convert` 

foo = Foo("hello", 23, 1.5)
@show foo, typeof(foo)
@show fieldnames(Foo)
@show foo.bar, foo.baz, foo.qux

struct C
    A::Array{Float64, 2}
end

c = C(rand(1,1))

@show c.A
@show rand!(c.A) # can mutate mutable fields

# mutable struct: allocated on heap, stable memory addresses

mutable struct Bar
   baz
   qux::Float64
end


# parametric type
#     ⋅ take parameters to instantiate a family of types

struct Point{T}
   x::T
   y::T
end

function norm(p::Point{T}) where T <: Real
    sqrt(p.x^2 + p.y^2)
end


p = Point{Float64}(2,3)

@show (Point{Float64} <: Point) == true
@show norm(p)
@show typeof(Point(2,3)) # infers Int64

Base.show(io::IO, p::Point{T}) where T <: Real = print(io, "point: (", p.x, ", ", p.y, ")")
@show p

In [ ]:
# https://docs.julialang.org/en/v1/manual/constructors/#man-constructors-1
#

struct Baz
   bar
   baz
end

# outer contructor
#    ⋅ can only create instance by calling another constructor

Baz(x) = Baz(x, x)
Baz() = Baz(0)

@show Baz(1, 2)
@show Baz(1)
@show Baz()

# inner constructor
#     ⋅ enforce invariance, allows constructing self-referential object
#     ⋅ declared inside `struct` block, access to `new`
#     ⋅ if defined, no default constructor is provided

struct OrderedPair
   x::Real
   y::Real
   OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

@show OrderedPair(1, 2)

# parametric constructor

In [ ]:
# https://docs.julialang.org/en/v1/manual/methods/
#

# Function
#     ⋅ maps tuple of arguments to a return value
#     ⋅ implements a concept
# Method
#     ⋅ one possible behavior of a function
#     ⋅ can be annotated to indicate the types of args, and their number
#     ⋅ the most specific type of method is applied, executed by `dispatch`
#     ⋅ No automatic casting or conversion of function args is performed!
f(x,y) = 0
f(x::Float64, y::Float64) = 1
f(x::Number, y::Number) = 2
@show f(1.,2.)
@show f(1,2.2)
@show methods(f)

# parametric methods

# aplies when both args are of same type 
same_type(x::T, y::T) where {T} = true
# catch all, cover other cases
same_type(x,y) = false
@show same_type(1,2), same_type(1,2.0)

myappend(v::Vector{T}, x::T) where {T} = [v..., x]
@show myappend([1,2,3], 4)

# redefining/add new methods
#     changes don't take effect immediately
@show Base.invokelatest(f, 1, 2)

In [ ]:
# https://docs.julialang.org/en/v1/manual/modules/
# 
module MyModule

using Pkg; Pkg.activate(".")
using PyPlot: plot
import Base.show

# include("file1.jl")

export MyType, foo

struct MyType
    x
end

bar(x) = 2x # private
foo(a::MyType) = bar(a.x) + 1

end

# Core, Base, Main
# a baremodule only imports Core

In [ ]:
# https://docs.julialang.org/en/v1/manual/arrays/
# 
@show zeros(Float32, 2,3)
@show ones(Int8, 2,3)
@show rand(2,3)
@show randn(2,3)
@show [1:2, 4:5] # creates 1-dim array containing values
@show [1:2; 4:5] # vcat or cat(A...; dims=1)
@show [1:2 4:5]  # hcat or cat(A...; dims=2)

# comprehension
x = rand(8)
# take average of neighbors!
@show [1/4*x[i-1] + 1/2*x[i] + 1/4*x[i+1] for i=2:length(x)-1] 

# generator (comprehension without [], ... no alloaction)
@show sum(1/n^2 for n = 1:1000)

# indexing
A = reshape(collect(1:16), (2,2,2,2))
@show A[1,1,1,1] # scalar index
@show A[[1,2], 1, [1,2], 1]  # mixed scalar/vector index
@show A[[1 2; 1 2]] # matrix index

# iteration
for i in eachindex(A)
    @show i, A[i]
end

# broadcast
a = rand(2,1); A = rand(2,3)
@show repeat(a,1,3) + A == broadcast(+, a, A) == (a .+ A)


# DenseArray: col-major contiguously allocated
#     1d: Vector, 2d: Matrix
# SubArray: created by `view` 

In [ ]:
# https://docs.julialang.org/en/v1/manual/performance-tips/
#

# perf with @time

# Type declarations
#    ⋅ declare type for attributes in a struct!

# break functions into multiple definitions (multiplex by type of arg)

# access arrays in memory order, along columns
@show [1 2; 3 4][:]

# pre-allocated array for max efficiency
xinc(x) = [x, x+1, x+2]
function loopinc()
    y = 0
    for i = 1:10^7
        ret = xinc(i)
        y += ret[2]
    end
    y
end

function xinc!(ret::AbstractVector{T}, x::T) where T
    ret[1] = x
    ret[2] = x + 1
    ret[3] = x + 2
    nothing
end

function loopinc_prealloc()
    ret = Vector{Int}(undef, 3)
    y = 0
    for i = 1:10^6
        xinc!(ret, i)
        y += ret[2]
    end
    y
end

@show @time loopinc()
@show @time loopinc_prealloc()

# use views for slices
fcopy(x) = sum(x[2:end-1])
@views fview(x) = sum(x[2:end-1])
x = rand(10^6)
@show @time fcopy(x)
@show @time fview(x)

# @code macros
f(x) = sin.(cos.(x))
@show @code_lowered f([1,2,3])

In [ ]:

# @inbounds
# @fastmath  floating point optimization for reals
# @simd      iterations are indep and can be re-ordered

@noinline function inner(x, y)
    s = zero(eltype(x))
    for i=eachindex(x)
        @inbounds s += x[i]*y[i]
    end
    return s
end

@noinline function innersimd(x, y)
    s = zero(eltype(x))
    @simd for i = eachindex(x)
        @inbounds s += x[i] * y[i]
    end
    return s
end

function timeit(n, reps)
    x = rand(Float32, n)
    y = rand(Float32, n)
    s = zero(Float64)
    time = @elapsed for j in 1:reps
        s += inner(x, y)
    end
    println("GFlop/sec        = ", 2n*reps / time*1E-9)
    time = @elapsed for j in 1:reps
        s += innersimd(x, y)
    end
    println("GFlop/sec (SIMD) = ", 2n*reps / time*1E-9)
end

timeit(1000, 1000)

In [ ]:
# https://docs.julialang.org/en/v1/manual/workflow-tips/
#

# Use Revise!